In [2]:
import pandas as pd
import numpy as np
import spacy

In [4]:
nlp = spacy.load('en')

In [5]:
doc = nlp("Tea is healthy and calming, don't you think?")

In [7]:
print(f"Token \t\tLemma \t\tStopword".format('Token','Lemma','Stopword'))
print('-'*40)
for token in doc:
    print(f'{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}')

Token 		Lemma 		Stopword
----------------------------------------
Tea		tea		False
is		be		True
healthy		healthy		False
and		and		True
calming		calm		False
,		,		False
do		do		True
n't		not		True
you		-PRON-		True
think		think		False
?		?		False
